In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
past_sales = pd.read_csv('/kaggle/input/rossmann-store-sales/train.csv')
store = pd.read_csv('/kaggle/input/rossmann-store-sales/store.csv')
past_sales['StateHoliday'] = past_sales['StateHoliday'].replace({0: '0'})

In [ ]:
store_imputed = store.copy()

avg_comp_dist = store_imputed['CompetitionDistance'].mean()
store_imputed['CompetitionDistance'] = store_imputed['CompetitionDistance'].fillna(avg_comp_dist)

store_imputed['CompetitionOpenSince'] = store_imputed['CompetitionOpenSinceYear'].astype(str) + '_' + store_imputed['CompetitionOpenSinceMonth'].astype(str)
store_imputed['CompetitionOpenSince'] = store_imputed['CompetitionOpenSince'].replace({'nan_nan': "Unknown"})
store_imputed = store_imputed.drop(['CompetitionOpenSinceYear', 'CompetitionOpenSinceMonth'], axis=1)

store_imputed['Promo2Since'] = store_imputed['Promo2SinceWeek'].astype(str) + '_' + store_imputed['Promo2SinceYear'].astype(str)
store_imputed['Promo2Since'] = store_imputed['Promo2Since'].replace({'nan_nan': "Unknown"})
store_imputed = store_imputed.drop(['Promo2SinceWeek', 'Promo2SinceYear'], axis=1)

store_imputed['PromoInterval'] = store_imputed['PromoInterval'].fillna('Unknown')
store_imputed.isna().sum()

In [ ]:
past_sales_merged = pd.merge(left=past_sales,
                             right=store_imputed,
                             on=['Store'],
                             how='left')
past_sales.shape, past_sales_merged.shape

In [ ]:
past_sales_merged['Date'] = pd.to_datetime(past_sales_merged['Date'], format='%Y-%m-%d')
past_sales_merged['month'] = past_sales_merged['Date'].dt.strftime('%b')
past_sales_merged['dayofmonth'] = past_sales_merged['Date'].dt.strftime('%d').astype(int)

In [ ]:
cols2drop = ['Store', 'Date', 'Customers', 'CompetitionOpenSince', 'Promo2Since']

dummies = pd.get_dummies(past_sales_merged.drop(cols2drop, axis=1), drop_first=True)
past_sales_merged.shape, dummies.shape